In [1]:
import logging
from hydra import compose, initialize

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

with initialize(version_base=None, config_path="./config"):
    cfg = compose(config_name="properties")

In [2]:
from omegaconf import DictConfig
from openai import RateLimitError
from langchain.chains.base import Chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_core.runnables import ConfigurableField
from langchain_core.messages import HumanMessage, SystemMessage

def build_chat_model(cfg: DictConfig):
    model = AzureChatOpenAI(
        **cfg.llm.openai
    ).configurable_alternatives(
        ConfigurableField(id="llm_type"),
        default_key="openai"
    )
    return model

MAX_ATTEMPT = 3

model = build_chat_model(cfg)
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}.")
output_parser = StrOutputParser()

# 1. LLM Simple Usage 
# # results in underlying API call of:
# # openai.AzureOpenAI(..).chat.completions.create(..., logprobs=True)

# messages = [
#     (
#         "system",
#         "You are a helpful translator. Translate the user sentence to French.",
#     ),
#     ("human", "I love programming."),
# ]
# ai_message = model.invoke(messages, logprobs=True)

# 2. LLM Chain Usage with LCEL

# To follow the steps along:

# 1. We pass in user input on the desired topic as {"topic": "stock"}
# 2. The prompt component takes the user input, which is then used to construct a PromptValue after using the topic to construct the prompt.
# 3. The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a ChatMessage object.
# 4. Finally, the output_parser component takes in a ChatMessage, and transforms this into a Python string, which is returned from the invoke method.

chain = (
    prompt
    | model.with_retry(
        retry_if_exception_type=(RateLimitError,),
        stop_after_attempt=MAX_ATTEMPT,
        wait_exponential_jitter=True
    ) 
    | output_parser
)
response = chain.invoke({"topic": "stock"})

In [24]:
# Key Methods - langchain.runnables.base.Runnable

#     - **invoke/ainvoke**: Transforms a single input into an output.
#     - **batch/abatch**: Efficiently transforms multiple inputs into outputs.
#     - **stream/astream**: Streams output from a single input as it's produced.
#     - **astream_log**: Streams output and selected intermediate results from an input.

#     Built-in optimizations:

#     - **Batch**: By default, batch runs invoke() in parallel using a thread pool executor.
#       Override to optimize batching.

#     - **Async**: Methods with "a" suffix are asynchronous. By default, they execute
#       the sync counterpart using asyncio's thread pool.
#       Override for native async.

#     **RunnableSequence** invokes a series of runnables sequentially, with
#     one Runnable's output serving as the next's input. Construct using
#     the `|` operator or by passing a list of runnables to RunnableSequence.

#     **RunnableParallel** invokes runnables concurrently, providing the same input
#     to each. Construct it using a dict literal within a sequence or by passing a
#     dict to RunnableParallel.

from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

# Runnable Sequence constructed using the | operator
sequence = RunnableLambda(lambda x: x+1) | RunnableLambda(lambda x: x*2)
print(sequence.invoke(1))
print(sequence.batch([1, 2, 3], config={"max_concurrency": 5}))

# A sequence that contains a RunnableParallel constructed using a dict parallel
sequence = RunnableLambda(lambda x: x+1) | {
    "mul_2": RunnableLambda(lambda x: x*2),
    "mul_5": RunnableLambda(lambda x: x*5)
}
print(sequence.invoke(1))
print(sequence.input_schema.schema())
print(sequence.output_schema.schema()) # RunnableParallel

sequence = RunnableLambda(lambda x: x+1) | RunnableParallel(
    {
        "mul_2": lambda x: x*2,
        "mul_5": RunnableLambda(lambda x: x*5),
        "pass": RunnablePassthrough()
    }
)
print(sequence.invoke(1))
print(sequence.input_schema.schema())
print(sequence.output_schema.schema()) # RunnableParallel

# All Runnables expose additional methods that can be used to modify their behavior
# (e.g., add a retry policy, add lifecycle listeners, make them configurable, etc.).

import random
import random

def add_one(x: int) -> int:
    return x + 1


def buggy_double(y: int) -> int:
    '''Buggy code that will fail 70% of the time'''
    if random.random() > 0.3:
        print('This code failed, and will probably be retried!')  # noqa: T201
        raise ValueError('Triggered buggy code')
    return y * 2

sequence = (
    RunnableLambda(add_one) |
    RunnableLambda(buggy_double).with_retry( # Retry on failure
        stop_after_attempt=10,
        wait_exponential_jitter=False
    )
)
print(sequence.invoke(2))

# RunnableParallel with itemgetter when given multiple input keys

from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda

prompt = """original number: {num}
Add one nmber: {num_1}"""
prompt = ChatPromptTemplate.from_template(prompt)

sequence = (
    RunnableParallel(
        {
            "num_1": itemgetter("num") | RunnableLambda(lambda x: x+1),
            "num": itemgetter("num")
        }
    )
    | prompt
)

prompt_val = sequence.invoke({"num": 1, "UNUSED_PARAM": 999})
print(prompt_val)

4
[4, 6, 8]
{'mul_2': 4, 'mul_5': 10}
{'title': 'RunnableLambdaInput'}
{'title': 'RunnableParallel<mul_2,mul_5>Output', 'type': 'object', 'properties': {'mul_2': {'title': 'Mul 2'}, 'mul_5': {'title': 'Mul 5'}}}
{'mul_2': 4, 'mul_5': 10, 'pass': 2}
{'title': 'RunnableLambdaInput'}
{'title': 'RunnableParallel<mul_2,mul_5,pass>Output', 'type': 'object', 'properties': {'mul_2': {'title': 'Mul 2'}, 'mul_5': {'title': 'Mul 5'}, 'pass': {'title': 'Pass'}}}
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
This code failed, and will probably be retried!
6
